### Setup

We will be using the BERT model and tokenizer from Hugging Face's transformers library. Additionally, the regular expressions module is used to preprocess our protein sequence to make it compatible with the model's expected input format.

We then load a tokenizer for protein sequences from the model hub, and then load the pre-trained protein sequence BERT model which acts as our encoder.

In [1]:
from transformers import BertModel, BertTokenizer, BertConfig
import re
import torch as torch
import torch.onnx

tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
model = BertModel.from_pretrained("Rostlab/prot_bert")

C:\tools\Manim\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = BertConfig.from_pretrained("Rostlab/prot_bert")


### Preprocessing

We'll define a sample protein sequence and preprocess it. Specifically, the U, Z, O, and B amino acids are much less common. For the purposes of standardizing input for the model, they are replaced by "X".

In [3]:
sequence_Example = "A E T C Z A O A E T C Z A O"
sequence_Example = re.sub(r"[UZOB]", "X", sequence_Example)

### Tokenizing

Now, we tokenize the sequence to convert it into a format that the BERT model understands. We use PyTorch tensors (`pt`) as that's the format the model expects.

In [4]:
sequence_Example = ["A E T C Z A O A E T C Z A O", "A B C D E F G", "D E F G G G G X X"]
tokens = tokenizer(sequence_Example,padding=True,truncation=True, max_length=512,return_tensors='pt')
tokens

{'input_ids': tensor([[ 2,  6,  9, 15, 23, 28,  6, 29,  6,  9, 15, 23, 28,  6, 29,  3],
        [ 2,  6, 27, 23, 14,  9, 19,  7,  3,  0,  0,  0,  0,  0,  0,  0],
        [ 2, 14,  9, 19,  7,  7,  7,  7, 25, 25,  3,  0,  0,  0,  0,  0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]])}

In [17]:
lengths = [sum(1 for token in seq if token != 0) for seq in tokens["attention_mask"]]
lengths

[16, 9, 11]

In [11]:
len(tokens['input_ids'][0])

16

### Fetching Embeddings from BERT

With the processed sequence, fetch the embeddings or representations from the BERT model. These embeddings can be utilized for further analysis, such as classification.

In [14]:

encodings=model(**tokens).last_hidden_state.mean(dim=1)

In [15]:
encodings


tensor([[ 0.0596,  0.0577, -0.0590,  ..., -0.0516, -0.0697,  0.0888],
        [ 0.0621,  0.0518, -0.0627,  ..., -0.0475, -0.0510,  0.0878],
        [-0.0183,  0.0620, -0.1026,  ..., -0.0218, -0.0510,  0.0492],
        [ 0.0062,  0.0601, -0.0934,  ..., -0.0455, -0.0555,  0.0700],
        [-0.0242,  0.1097, -0.1192,  ..., -0.0284, -0.0762,  0.0547]],
       grad_fn=<MeanBackward1>)

In [55]:
encodings[1]

tensor([ 0.0639,  0.0582, -0.0569,  ..., -0.0532, -0.0593,  0.0873],
       grad_fn=<SelectBackward0>)

In [ ]:
torch.onnx.export(model,             
                      args=(tokens['input_ids'],),
                      f="protein_embedding.onnx",   
                      input_names=['input_ids'],   
                      output_names=['protein_embeddings'], 
                      opset_version=11)     

In [6]:
import pandas as pd


token_ids = tokens['input_ids'].numpy()  # Convert to numpy array
df = pd.DataFrame(token_ids)


file_path = r'Data\protein\example.csv'

df.to_csv(file_path, index=False, header=False)